# Create a short text clustering system using AWS SageMaker jumpstart pre-trained transformer models 

1. [Introduction](#Introduction)  
2. [Setup](#Setup)
3. [Create a model for text embeddings from the Jumpstart solutions library of models](#Create-a-model-for-text-embeddings-from-the-Jumpstart-solutions-library-of-models)
4. [Data pre-processing](#Data-pre-processing)
5. [Create phrase (sentence) embeddings](#Create-phrase-(sentence)-embeddings)
6. [Cluster phrases (sentences)](#Cluster-phrases-(sentences))
7. [Automatic cluster labeling](#Automatic-cluster-labeling)
8. [Batch process the entire dataset](#Batch-process-the-entire-dataset)

## Introduction

In this notebook we demonstrate how you can cluster short text (phrases) using the pre-trained transformer models on [AWS SageMaker Jumpstart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html). Here we will demonstrate the use of a transformer model called [bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli). The model is used to create an embedding of phrases that we will then use to cluster such phrases.

## Setup

Let's start by updating the required packages i.e. SageMaker Python SDK, pandas, numpy, etc.

In [2]:
!pip install boto3 jsonlines seaborn

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


# **Note: Restart the notebook's kernel after installing the above packages.**

In [3]:
import boto3
import sagemaker
import json
import re
import os

from sagemaker import get_execution_role

import pandas as pd
import numpy as np
import math

import nltk
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

from sklearn.manifold import TSNE
from sklearn.cluster import SpectralClustering

We use NLTK library to help us with the pre-processing of the data

In [4]:
session = boto3.Session()
sagemaker_execution_role = get_execution_role()
s3 = session.resource('s3')

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Create a model for text embeddings from the Jumpstart solutions library of models

We will use one the text embedding [models available in SageMaker jumpstart](https://sagemaker.readthedocs.io/en/v2.129.0/doc_utils/pretrainedmodels.html)

#### Chose a model for Inference

In [6]:
#We choose the tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1 as the default model since it is better suited for phrase analysis

model_id, model_version = (
    "tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1", 
    "*")

You can continue with the default model, or can choose a different model from the dropdown generated upon running the next cell. A complete list of SageMaker pre-trained models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).

In [7]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models, list_jumpstart_tasks

# Retrieves all text embedding models.
filter_value = "task == tcembedding"
tcembedding_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=tcembedding_models,
    value=model_id,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [8]:
display(model_dropdown)

Dropdown(description='Select a model', index=33, layout=Layout(width='max-content'), options=('mxnet-tcembeddi…

In [9]:
# model_version="*" fetches the latest version of the model
model_id, model_version = model_dropdown.value, "*"

In [10]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"jumpstart-example-infer-{model_id}")

### Create the model from the selected model_id, model_version

In [11]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor

inference_instance_type = "ml.m5.xlarge"  #You can change the instance according to your needs

# Retrieve the inference docker container uri. This is the base Tensorflow container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)


# Retrieve the model uri. This includes the model and model parameters.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)


# Create the SageMaker model instance
embedding_model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=model_uri,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    role=sagemaker_execution_role,
    predictor_cls=Predictor,
    name=model_name,
)

## Data pre-processing

For this demonstration we will use a dataset made up of the blog titles for each blog published by AWS from 2004 until late 2022. We use the blog titles to cluster them and assign a topic to each cluster

The text is pre-processed with the following steps:

* Set category string to lowercase
* Replace acronyms with actual words 
* Replace special word-bound characters such as / and - (i.e.: imagenes/videos, cerveza-vino) to get separate words.
* Eliminate explanations between parenthesis
* Remove any other non-word characters from sentence
* Split sentence into tokens
* Singularize each token

In [12]:
blogs_df = pd.read_csv('aws_blog_titles.csv', header=None, names=['URL', 'Title'])

aws_acronyms_df = pd.read_csv('acronyms.csv', header=None, delimiter=';', names=['acronym', 'meaning'])

In [13]:
blogs_df

,URL,Title
0,https://aws.amazon.com/blogs/aws/10000_sheep_col/,10 000 Sheep Collaborative Art Project
1,https://aws.amazon.com/blogs/apn/10-best-pract...,10 Best Practices to Help Partners Build AWS Q...
2,https://aws.amazon.com/blogs/architecture/ten-...,10 Things Serverless Architects Should Know
3,https://aws.amazon.com/blogs/apn/10-years-of-s...,10 Years of Success AWS and Alert Logic
4,https://aws.amazon.com/blogs/apn/10-years-of-s...,10 Years of Success AWS and Appian
...,...,...
23195,https://aws.amazon.com/blogs/media/reinvent-bo...,re Invent bonus content M E focused sessions o...
23196,https://aws.amazon.com/blogs/opensource/reinve...,re Invent open source highlights Week 1
23197,https://aws.amazon.com/blogs/startups/redbus-b...,redBus Building a Data Platform with AWS Apach...
23198,https://aws.amazon.com/blogs/security/s2n-and-...,s2n and Lucky 13


In [14]:
#We transform acronyms to their actual meaning since the transformer may not be aware of them (as it was not trained in this specific vocabulary)

aws_acronyms_df

,acronym,meaning
0,Amazon ES,Amazon Elasticsearch Service
1,AMI,Amazon Machine Image
2,API,Application Programming Interface
3,AI,Artificial Intelligence
4,ACL,Access Control List
...,...,...
120,VPN,Virtual Private Network
121,VLAN,Virtual Local Area Network
122,VDI,Virtual Desktop Infrastructure
123,VPG,Virtual Private Gateway


In [15]:
blogs_df = blogs_df.drop_duplicates(subset=['Title']).reset_index()
blogs_df = blogs_df.drop(columns=['index', 'URL'])

In [16]:
#For eficiency only take sample_size titles at random

sample_size = 1000
blogs_df_sample = blogs_df.sample(n=sample_size)

In [17]:
titles = blogs_df_sample['Title'].tolist()

In [18]:
lemmatized = []
for title in titles:
    sentence = title.lower()
    sentence = re.sub(r'[^a-zA-Z0-9_-áéíóúñ ]', r'', sentence)  #remove extraneous characters (maybe a different encoding)
    lemmatized.append(sentence)

In [19]:
lemmatized[:10]

['nova brings data science courses to the usmc with aws educate',
 'cloud native application monitoring for aws',
 'sharing matlab applications on aws using the matlab web app server',
 'amazon elastic file system shared file storage for amazon ec2',
 'duplicating infrastructure on aws',
 'amazon kinesis analytics process streaming data in real time with sql',
 'how to enable 360 degree analytics and innovate faster on aws with datavard glue for sap',
 'amazon monitron a simple and cost effective service enabling predictive maintenance',
 'new aws partner program launches and updates announced at re invent 2020',
 'online tech talk april 23 persistent storage for containers with amazon efs']

## Create phrase (sentence) embeddings

In [20]:
# These functions are used to query the endpoint and parse the response

def query(model_predictor, text):
    """Query the model predictor."""

    encoded_text = json.dumps(text).encode("utf-8")

    query_response = model_predictor.predict(
        encoded_text,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
    )
    return query_response


def parse_response(query_response):
    """Parse response and return the embedding."""

    model_predictions = json.loads(query_response)
    embedding = model_predictions["embedding"]
    return embedding

### Deploy the selected model to an endpoint for real time inference

In [21]:
# deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model_predictor = embedding_model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=model_name,
)

-----!

### Generate embeddings

We use the deployed model to generate the embeddings for each of the titles in our sample dataset

In [22]:
#model_predictor = Predictor('jumpstart-example-infer-tensorflow-tcem-2023-01-19-23-23-44-619')  #Specifiy endpoint name in case you wanna use an already deployed endpoint

In [23]:
%%time
sentence_vectors = [parse_response(query(model_predictor, title)) for title in lemmatized]

CPU times: user 2.37 s, sys: 142 ms, total: 2.52 s
Wall time: 4min 59s


In [24]:
encoded_titles_df = pd.DataFrame(sentence_vectors)
encoded_titles_df['blog_title_lemmatized'] = lemmatized

In [25]:
encoded_titles_df

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,blog_title_lemmatized
0,-1.259244,0.102127,0.089923,-0.820289,-0.223537,-0.432897,0.799307,-0.674757,-0.252520,-0.272772,...,0.141702,-0.392942,0.119546,-0.386988,-0.568477,0.205832,-0.170402,-1.025766,0.198024,nova brings data science courses to the usmc w...
1,-0.692962,0.259012,-0.748741,-0.866656,-0.901526,-0.997768,0.377633,-0.053566,-0.503752,0.144736,...,0.114172,0.522053,0.834067,-0.771977,-0.466612,0.195516,-0.715755,0.326673,-0.530868,cloud native application monitoring for aws
2,-0.652742,0.126623,0.094145,-0.545012,-0.761059,-0.479529,-0.119898,0.001302,-0.347351,0.485530,...,0.158906,-0.314114,0.502422,-0.112350,-0.143338,-0.790585,-0.104163,-0.003831,-0.414520,sharing matlab applications on aws using the m...
3,0.157903,-1.218128,-0.358249,-0.143645,-0.785935,0.111689,0.451628,-0.040183,0.232529,0.813196,...,0.574929,-0.595977,0.733360,0.228505,0.484546,0.407535,-0.656069,0.465197,0.631824,amazon elastic file system shared file storage...
4,-0.199768,0.498280,-0.096403,-1.235060,-0.164219,-0.747414,-0.280207,0.717804,-1.201374,0.192102,...,-0.422836,0.048728,0.729825,-0.974279,0.452009,0.646301,-0.731122,0.526780,-0.412448,duplicating infrastructure on aws
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.190524,0.190522,0.247199,-0.909789,-0.532487,-0.958266,-0.366771,-0.469838,-0.685504,-0.749760,...,-0.482015,0.358346,0.935664,-0.944232,0.374791,0.138823,0.717717,-0.581709,0.456586,aws week in review august 25 2014
996,-0.524172,-0.508515,0.474869,-1.093429,-1.255689,-0.688424,0.898723,0.750922,-1.154738,1.334882,...,0.765626,-0.240315,0.339106,-0.142825,0.226154,-0.350119,-0.670260,-0.665765,-0.013468,optimizing performance for users in china with...
997,-0.404731,0.581385,-0.491551,-0.588944,-0.035981,-0.529826,0.459939,0.522350,-0.774718,0.081250,...,0.718470,-0.013323,1.003814,-0.764670,-0.364241,0.231567,-0.153268,-0.106582,-1.071888,aws accelerator for citrix migrate or deploy x...
998,0.307507,0.646797,0.672350,-0.879446,-0.829220,-0.173903,0.130434,-0.228525,0.172912,-0.866439,...,-0.857214,-0.324904,1.445891,-0.198214,-0.104684,0.646890,-0.060024,-0.384143,0.098519,in case you missed it september 2019 top blog ...


## Cluster phrases (sentences)

Spectral clsutering is a clustering algorithm based on graph theory. Spectral clustering uses information from the eigenvalues (spectrum) of the Laplacian matrix built from the graph or the data set to create groups (clusters) of data. Spectral clustering requires a measures of affinity between data points, for this application we use cosine affinity because we are interested in sentences that lie near to each other but also with "similar meaning".

In [26]:
n_clusters = 20

clustering_model = SpectralClustering(n_clusters=n_clusters, n_init=100, affinity='cosine', n_neighbors=10, assign_labels="kmeans", random_state=0)
embeddings = encoded_titles_df[encoded_titles_df.columns[0:-1]]
encoded_titles_df['cluster'] = clustering_model.fit_predict(embeddings)

In [27]:
clusters_titles = encoded_titles_df[['blog_title_lemmatized', 'cluster']]

In [28]:
clusters_titles

,blog_title_lemmatized,cluster
0,nova brings data science courses to the usmc w...,9
1,cloud native application monitoring for aws,17
2,sharing matlab applications on aws using the m...,0
3,amazon elastic file system shared file storage...,5
4,duplicating infrastructure on aws,17
...,...,...
995,aws week in review august 25 2014,6
996,optimizing performance for users in china with...,8
997,aws accelerator for citrix migrate or deploy x...,17
998,in case you missed it september 2019 top blog ...,6


## Automatic cluster labeling

### Use TF-IDF for finding the keywords in each of our clusters

Text Frequency - Inverse Document Frequency is an NLP technique used to find the most relevant terms in set of documents (phrases in our case). From each cluster we extract its most relevant terms (nouns only) according to TF-IDF and use those as labels/categories for that cluster

In [36]:
clusters = [clusters_titles.loc[clusters_titles.cluster == i, 'blog_title_lemmatized'].to_list() for i in range(0,n_clusters)]

In [37]:
clusters_tf_idf = []
clusters_tf_idf_terms = []
clusters_tags = []
clusters_keywords_tf_idf = []
tf_idf_threshold = 0.2

for cluster in clusters:

    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cluster)
    tf_idf_df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    tf_idf_df = tf_idf_df.sort_values('TF-IDF', ascending=False)
    
    clusters_tf_idf.append(tf_idf_df)
    
    cluster_tf_idf_terms = list(tf_idf_df.loc[tf_idf_df['TF-IDF'] > tf_idf_threshold].index.values)
    clusters_tf_idf_terms.append(cluster_tf_idf_terms)
    
    tags = nltk.pos_tag(cluster_tf_idf_terms)
    clusters_tags.append(tags)
    
    keywords = [tag[0] for tag in tags if tag[1] in ['NN', 'NNS'] and tag[0] not in ['aws', 'amazon']]
    clusters_keywords_tf_idf.append(keywords)

In [38]:
clusters_idf = []

for cluster in clusters:

    cv = CountVectorizer() 
    word_count_vector = cv.fit_transform(cluster)

    tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
    tfidf_transformer.fit(word_count_vector)

    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
    df_idf['word'] = cv.get_feature_names()

    df_idf = df_idf.sort_values(by='idf_weights')
    clusters_idf.append(df_idf)

In [39]:
clusters_keywords_tf_idf

[['matlab', 'applications', 'app'],
 ['gen', 'adopters', 'problems'],
 ['model', 'register', 'security'],
 ['tool', 'risks', 'manage', 'systems'],
 ['messaging'],
 ['file', 'storage', 'system'],
 ['september', 'week', 'review'],
 ['connectivity', 'options', 'gigabit'],
 ['quality', 'call', 'connect', 'detection', 'time', 'opensearch'],
 ['educate', 'nova', 'brings', 'courses', 'science', 'data'],
 ['service', 'monitron', 'simple', 'maintenance', 'cost'],
 ['service', 'database'],
 ['uploads', 's3'],
 ['process', 'streaming', 'time', 'analytics', 'kinesis'],
 ['reports', 'soc'],
 ['sydney', 'university', 'technology', 'stroke', 'rehabilitation', 'robots'],
 ['jenkins', 'party', 'create', 'source', 'projects', 'control'],
 ['application', 'monitoring', 'cloud'],
 ['efs', 'talk', 'tech', 'online', 'containers', 'storage'],
 ['program', 'updates', 'partner']]

In [40]:
clusters_idf[0][:20]

,idf_weights,word
aws,1.117783,aws
the,1.492476,the
for,1.750306,for
sdk,1.944462,sdk
with,1.944462,with
and,2.098612,and
to,2.386294,to
using,2.504077,using
android,2.791759,android
on,2.791759,on


In [41]:
clusters_keywords_idf = []
stop_words = stopwords.words('english')

clusters_idf_words = [cluster['word'].to_list()[0:10] for cluster in clusters_idf]
s = [ word for word in stop_words if word != 're'] #Remove stopwords but the word re (for re: invent)

for cluster in clusters_idf_words:
    tags = nltk.pos_tag(cluster)
    words = [ tag[0] for tag in tags if tag[1] in ['NN', 'NNS'] and tag[0] not in ['aws', 'amazon']]
    
    clusters_keywords_idf.append(",".join(words))

In [42]:
clusters_keywords_idf

['sdk,android',
 '',
 'security,summit',
 'manager,systems,management',
 '',
 'instances,instance',
 'review,week,september,part',
 'partners',
 'service,rekognition',
 'data',
 'cloud,data,management',
 'rds,database',
 'cloudformation,update',
 'data,redshift,dynamodb,s3',
 'source,time',
 '',
 'sagemaker,model,inference,data',
 'workloads',
 '',
 're,invent,guide']

## Batch process the entire dataset

In this section we will create batch processing jobs to process the entire dataset (roughly 24K titles)

In [43]:
import io
import jsonlines

from sagemaker.s3 import S3Downloader,S3Uploader,s3_path_join

n_clusters = 20

In [44]:
bucket_name = 'unsupervised-phrase-clustering-with-sagemaker' #<REPLACE_WITH_YOUR_BUCKET_NAME>
s3_prefix = 'text-clustering-with-transformers/data'

#### Chose a model for Inference

In [45]:
#We choose the tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1 as the default model since it is better suited for phrase analysis

model_id, model_version = (
    "tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1", 
    "*")

You can continue with the default model, or can choose a different model from the dropdown generated upon running the next cell. A complete list of SageMaker pre-trained models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).

### Create the model from the selected model_id, model_version

In [46]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"jumpstart-example-infer-gpu-{model_id}")

In [47]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor

batch_transform_instance_type = "ml.g4dn.xlarge"

# Retrieve the inference docker container uri. This is the base Tensorflow container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=batch_transform_instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)


# Retrieve the model uri. This includes the model and model parameters.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)


# Create the SageMaker model instance
batch_transform_embedding_model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=model_uri,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    role=sagemaker_execution_role,
    name=model_name,
)

### Data preprocessing

In [48]:
blogs_df = pd.read_csv('aws_blog_titles.csv', header=None, names=['URL', 'Title'])
aws_acronyms_df = pd.read_csv('acronyms.csv', header=None, delimiter=';', names=['acronym', 'meaning'])
blogs_df = blogs_df.drop_duplicates(subset=['Title']).reset_index()
blogs_df = blogs_df.drop(columns=['index', 'URL'])
titles = blogs_df['Title'].tolist()

In [49]:
lemmatized = []
for title in titles:
    sentence = title.lower()
    sentence = re.sub(r'[^a-zA-Z0-9_-áéíóúñ ]', r'', sentence)  #remove extraneous characters (maybe a different encoding)
    lemmatized.append(sentence)

### Upload the pre-processed data to S3

In [50]:
batch_filename = 'aws_blog_titles.jsonl'

In [51]:
with open(batch_filename, "wb") as txt_file:
    for title in lemmatized:
        
        txt_file.write(json.dumps(title).encode("utf-8"))
        txt_file.write("\n".encode('utf-8'))

In [52]:
data_upload_path = s3_path_join("s3://",bucket_name,s3_prefix, 'raw')
print(f"Uploading data to {data_upload_path}")
data_uri = S3Uploader.upload(batch_filename, data_upload_path)
print(f"Uploaded data to {data_upload_path}")

Uploading data to s3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/raw
Uploaded data to s3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/raw


### Generate embeddings

In [53]:
# create transformer to run a batch job

output_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "embeddings")

batch_job = batch_transform_embedding_model.transformer(
    instance_count=1,
    instance_type=batch_transform_instance_type,
    strategy='SingleRecord',
    assemble_with='Line',
    output_path=output_path,
)

In [54]:
# Starts batch transform job and uses S3 data as input. Enable the logs and wait only if you pass a small number of samples (< 100).
# You can monitor your batch processing job from the SageMaker Console -> Inference -> Batch transform jobs
batch_job.transform(
    data=data_upload_path,
    content_type='application/x-text',    
    split_type='Line',
    logs=False,
    wait=False
)

In [55]:
#Download the results. 
#The batch transformation job (step above) must have finished before you can run this cell.
embedding_data_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "embeddings", batch_filename+'.out')
print(f"Downloading embeddings to .")
S3Downloader.download(embedding_data_path,'.')
print(f"Downloaded embeddings to .")

Downloaded embeddings to .


In [56]:
lines = []

with jsonlines.open(batch_filename+".out", mode='r') as reader:
    for obj in reader:
        lines.append(obj['embedding'])
        
results_df = pd.DataFrame(lines)
results_df['blog_title_lemmatized'] = lemmatized

In [57]:
results_df

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,blog_title_lemmatized
0,0.188292,-0.390401,-0.286212,-0.732304,-0.841209,-1.088366,-0.598213,1.023898,0.821023,-1.058815,...,-0.175997,0.245866,0.706102,0.783085,-0.823547,1.258304,-0.043126,-0.472476,-0.351324,10 000 sheep collaborative art project
1,-0.264925,-0.108307,0.688968,-0.680849,-0.351448,-0.899282,0.391024,-0.635701,0.246767,-0.395383,...,0.673714,0.167650,1.476131,0.409790,-0.050008,0.582545,0.279141,-0.205470,-0.820358,10 best practices to help partners build aws q...
2,0.371885,0.771379,1.069450,-1.008337,0.655574,-0.955685,-0.711391,0.625457,0.369845,-0.443280,...,-0.096941,-0.025879,0.967100,-0.283345,0.691451,0.882124,0.534032,-0.739388,0.378053,10 things serverless architects should know
3,-0.301370,0.847713,0.486485,-0.280148,-1.502466,-1.012401,-0.091017,0.317997,-0.872763,-0.809212,...,-0.392543,0.050554,0.026191,-0.436454,-0.320372,0.507621,-0.305170,-0.284728,-0.776441,10 years of success aws and alert logic
4,0.029871,0.295120,0.633014,-0.412160,-1.094922,-0.794995,0.185795,-0.116586,-0.548522,-1.379145,...,0.548648,0.010977,0.174071,-0.338245,-0.298892,0.599860,0.190257,-0.457519,-1.003187,10 years of success aws and appian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23125,-0.022962,1.256789,0.323558,-0.199724,0.179075,0.219997,0.139460,-0.184126,-0.289477,-0.732770,...,-0.750940,0.333212,0.500258,-0.642244,-0.251067,1.796083,-0.214922,-0.485310,-0.257823,re invent bonus content m e focused sessions o...
23126,0.700433,0.953094,1.036029,-0.781870,-0.111338,-0.478488,0.362508,-0.422137,0.382570,-0.086834,...,-0.269035,0.257387,0.684636,-0.135962,-0.267956,0.802172,-0.122769,-0.162395,0.466416,re invent open source highlights week 1
23127,0.096553,0.953412,0.493536,-1.050031,0.031639,-0.762166,-0.221962,0.420519,-0.328862,-0.170843,...,-0.729130,0.518689,0.360414,0.419243,-0.244598,0.524628,-0.373507,-0.994924,-0.055463,redbus building a data platform with aws apach...
23128,-0.662509,-0.092624,0.064623,-0.446625,-0.261067,-0.437387,0.701777,0.340038,-0.293806,-0.226297,...,0.881936,0.450375,0.289184,0.122812,0.063564,0.710250,-0.277890,-1.099730,0.753118,s2n and lucky 13


In [58]:
embeddings_filename = "blog_title_embeddings.csv"
results_df.to_csv(embeddings_filename, index=False)

In [59]:
embedding_upload_path = s3_path_join("s3://",bucket_name,s3_prefix, 'embeddings')
print(f"Uploading embeddings to {embedding_upload_path}")
data_uri = S3Uploader.upload(embeddings_filename, embedding_upload_path)
print(f"Uploaded embeddings to {embedding_upload_path}")

Uploading embeddings to s3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/embeddings
Uploaded embeddings to s3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/embeddings


### Cluster titles

In [60]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

In [61]:
sklearn_processor_spectral_clustering = SKLearnProcessor(framework_version='1.0-1',
                                                         role=sagemaker_execution_role,
                                                         instance_type='ml.m5.2xlarge',
                                                         instance_count=1)

In [62]:
output_destination = os.path.join('s3://', bucket_name, s3_prefix, "results", "clusters")

sklearn_processor_spectral_clustering.run(
    code="./scikit-sagemaker-clustering/SpectralClustering.py",
    inputs=[ProcessingInput(source=embedding_upload_path, destination="/opt/ml/processing/input")],
    outputs=[ProcessingOutput(output_name="titles_clusters", source="/opt/ml/processing/output", destination=output_destination)],
    arguments=["--n-clusters", str(n_clusters),
               "--n-init", "100",
               "--affinity", "cosine",
               "--n-neighbors", "10",
               "--assign-labels", "kmeans"
              ],
)


Job Name:  sagemaker-scikit-learn-2023-03-14-02-20-48-009
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/embeddings', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-230294632802/sagemaker-scikit-learn-2023-03-14-02-20-48-009/input/code/SpectralClustering.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'titles_clusters', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/results/clusters', 'LocalPath': '/opt/ml/processing/output'

In [63]:
#Download the results. 
#The batch clustering job (step above) must have finished before you can run this cell.

clusters_file = 'clustered_blog_titles_with_embeddings.csv'

clusters_data_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "clusters", clusters_file)
print(f"Downloading cluster data to .")
S3Downloader.download(clusters_data_path,'.')
print(f"Downloaded cluster data to .")

Downloaded cluster data to .


### Automatic cluster labeling

In [64]:
clusters_df = pd.read_csv(clusters_file)
clusters_titles = clusters_df[['blog_title_lemmatized', 'cluster_label']]

In [65]:
clusters_titles

,blog_title_lemmatized,cluster_label
0,10 000 sheep collaborative art project,19
1,10 best practices to help partners build aws q...,2
2,10 things serverless architects should know,13
3,10 years of success aws and alert logic,18
4,10 years of success aws and appian,18
...,...,...
23125,re invent bonus content m e focused sessions o...,15
23126,re invent open source highlights week 1,15
23127,redbus building a data platform with aws apach...,17
23128,s2n and lucky 13,19


In [66]:
clusters = [clusters_titles.loc[clusters_titles.cluster_label == i, 'blog_title_lemmatized'].to_list() for i in range(0, n_clusters)]

In [67]:
clusters_tf_idf = []
clusters_tf_idf_terms = []
clusters_tags = []
clusters_keywords_tf_idf = []
tf_idf_threshold = 0.2

for cluster in clusters:

    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cluster)
    tf_idf_df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    tf_idf_df = tf_idf_df.sort_values('TF-IDF', ascending=False)
    
    clusters_tf_idf.append(tf_idf_df)
    
    cluster_tf_idf_terms = list(tf_idf_df.loc[tf_idf_df['TF-IDF'] > tf_idf_threshold].index.values)
    clusters_tf_idf_terms.append(cluster_tf_idf_terms)
    
    tags = nltk.pos_tag(cluster_tf_idf_terms)
    clusters_tags.append(tags)
    
    keywords = [tag[0] for tag in tags if tag[1] in ['NN', 'NNS'] and tag[0] not in ['aws', 'amazon']]
    clusters_keywords_tf_idf.append(keywords)

In [68]:
clusters_keywords_tf_idf

[['blockchain', 'datasets', 'data'],
 ['alces', 'flight', 'series', 'sector', 'university', 'power', 'research'],
 ['starts', 'practices', 'help', 'customers'],
 ['visualizations', 'data', 'quicksight'],
 ['success', 'years'],
 ['turbines', 'drone', 'wind', 'inspection', 'ai', 'driven'],
 ['simpler', 'sam', 'experience', 'deployment', 'cli'],
 ['feedback', 'whitepapers', 'videos', 'articles', 'year'],
 ['backup', 'ways', 'plans', 'rules'],
 ['patient', 'health', 'part', 'building', 'digital'],
 ['innovation', 'years'],
 ['switzerland', 'isae', 'finma', 'attestation', 'type', 'report'],
 ['experience', 'style', 'shop', 'pytorch'],
 ['architects', 'things'],
 ['review', 'year', 'mongodb', 'compatibility'],
 ['catalog', 'session', 'compliance', 'security'],
 ['decade', 'iops', 'ebs'],
 ['things', 'compatibility', 'mongodb'],
 ['years', 'success'],
 ['art', 'project']]

In [69]:
clusters_df['categories'] = clusters_titles['cluster_label'].map(lambda i: clusters_keywords_tf_idf[i])

In [70]:
clusters_df.loc[clusters_df['cluster_label']==0, ['blog_title_lemmatized', 'cluster_label', 'categories']]

,blog_title_lemmatized,cluster_label,categories
48,22 new or updated open datasets on aws new pol...,0,"[blockchain, datasets, data]"
53,3 gain insights from complex data featuring 3m,0,"[blockchain, datasets, data]"
63,4 steps to train and deploy machine learning m...,0,"[blockchain, datasets, data]"
102,70 datasets inspire winning ideas to tackle op...,0,"[blockchain, datasets, data]"
105,890 by capgemini with aws powering business de...,0,"[blockchain, datasets, data]"
...,...,...,...
22916,what s around the turn in 2021 aws deepracer l...,0,"[blockchain, datasets, data]"
22972,why our customers love amazon machine learning...,0,"[blockchain, datasets, data]"
22989,why use docker containers for machine learning...,0,"[blockchain, datasets, data]"
22994,will spark power the data behind precision med...,0,"[blockchain, datasets, data]"


In [71]:
clusters_categories_file = 'aws_blog_titles_clusters_categories.csv'
clusters_df.to_csv(clusters_categories_file, index=False)

In [72]:
clusters_data_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "clusters")
print(f"Uploading clusters to {clusters_data_path}")
clusters_file_uri = S3Uploader.upload(clusters_categories_file, clusters_data_path)
print(f"Uploaded clusters to {clusters_data_path}")

Uploading clusters to s3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/results/clusters
Uploaded clusters to s3://unsupervised-phrase-clustering-with-sagemaker/text-clustering-with-transformers/data/results/clusters


### Visualize the clusters

In [73]:
cluster_sample_df = clusters_df.sample(n=1000).reset_index()
title_embeddings_sample = cluster_sample_df.iloc[:,:-3]
clusters_titles_sample = cluster_sample_df[['blog_title_lemmatized', 'cluster_label', 'categories']]
clusters_titles_sample['short_categories'] = clusters_titles_sample['categories'].map(lambda x: x[:2])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [74]:
clusters_titles_sample

,blog_title_lemmatized,cluster_label,categories,short_categories
0,new aws lambda scaling controls for kinesis an...,5,"[turbines, drone, wind, inspection, ai, driven]","[turbines, drone]"
1,high growth innovation powered by technology,9,"[patient, health, part, building, digital]","[patient, health]"
2,aws java dao integration project,18,"[years, success]","[years, success]"
3,aws online tech talks november 2017,18,"[years, success]","[years, success]"
4,icymi new stories updates and resources from a...,2,"[starts, practices, help, customers]","[starts, practices]"
...,...,...,...,...
995,announcing self service blacklisted address re...,8,"[backup, ways, plans, rules]","[backup, ways]"
996,improving daemon services in amazon ecs,16,"[decade, iops, ebs]","[decade, iops]"
997,improve your website availability with amazon ...,4,"[success, years]","[success, years]"
998,how to package cookbook dependencies locally w...,16,"[decade, iops, ebs]","[decade, iops]"


In [75]:
clusters_tsne = TSNE(perplexity=13, n_components=2, init='pca', n_iter=5000)
tsne_embeddings = clusters_tsne.fit_transform(title_embeddings_sample)
tsne_embeddings_df = pd.DataFrame(tsne_embeddings, columns=['x', 'y'])
tsne_embeddings_df['cluster'] = clusters_titles_sample['cluster_label']
tsne_embeddings_df['labels'] = clusters_titles_sample['categories']

In [ ]:
tsne_embeddings_df

In [ ]:
colors=[
    '#efaf50',
    '#a09934',
    '#e31ad9',
    '#cfcbb0',
    '#1224c9',
    '#669fa4',
    '#087274',
    '#787168',
    '#3e93cb',
    '#722823',
    '#c8784c',
    '#74ac48',
    '#c31033',
    '#5acc21',
    '#2ef8ba',
    '#c67ebe',
    '#805004',
    '#a8f43b',
    '#442d6d',
    '#9141ea',
]

fig, ax = plt.subplots(figsize=(30,30))
ax = sns.scatterplot(data=tsne_embeddings_df, x='x', y='y', hue='cluster', legend='full', palette=colors, ax=ax)
plt.show()

In [ ]:
tsne_embeddings_df[['cluster', 'labels']].drop_duplicates('cluster').sort_values('cluster')